In [2]:
%load_ext autoreload
%autoreload 2

# format all data.json files needed for running with LAUG files 
# 1. replace all text with text from laug text (TP & SD)
# 2. replace all text with mutated entities and change labels with mutated entities

In [3]:
import json

In [4]:
orig_fn = "/data/home/justincho/trippy-public-master/data/MULTIWOZ2.3/test_dials.json"
tp_fn = "data/laug_dst/TP/test_dials.json"
sd_fn = "data/laug_dst/SD/test_dials.json"

In [5]:
with open(orig_fn) as f: 
    orig = json.load(f)

with open(tp_fn) as f: 
    tp_test = json.load(f)
with open(sd_fn) as f: 
    sd_test = json.load(f)


In [6]:
list(orig.keys())[:2], list(tp_test.keys())[:2]

(['MUL0003.json', 'MUL0004.json'], ['SNG0073', 'PMUL4648'])

In [7]:
def normalize_keys(mwoz_dict): 

    new_dict = {} 
    for k, v in mwoz_dict.items(): 
        if ".json" not in k: 
            new_key = k + ".json"
        else: 
            new_key = k 
        new_dict[new_key] = v 

    return new_dict 

In [8]:
def test_match(orig_data, aug_data): 
    """ make sure that tp/sd testsets are the same as the original except for the new text"""
    orig_data = normalize_keys(orig_data)
    aug_data = normalize_keys(aug_data)
    assert orig_data.keys() == aug_data.keys(), "no match"
    print("Keys match")
    return 

In [9]:
test_match(orig, tp_test)
test_match(orig, sd_test)

Keys match
Keys match


In [10]:
orig_data = normalize_keys(orig)
sd_test = normalize_keys(sd_test)
tp_test = normalize_keys(tp_test)

In [11]:
orig_data["PMUL4648.json"].keys() 

dict_keys(['goal', 'log', 'new_goal'])

In [12]:
sd_test["PMUL4648.json"].keys() 


dict_keys(['goal', 'log', 'filename'])

In [13]:
sd_test["PMUL4648.json"]['log'][1] == orig_data["PMUL4648.json"]['log'][1]

False

In [14]:
# all that matters is 'text' and 'metadata' fields 

sd_sample = sd_test["PMUL4648.json"]['log'][5] 
orig_sample = orig_data["PMUL4648.json"]['log'][5]
if 'turn_id' in orig_sample:
    orig_sample.pop('turn_id')
if 'originalText' in sd_sample: 
    sd_sample.pop("originalText")
for k in orig_sample: 
    if k not in sd_sample: 
        print(k)

for k in sd_sample: 
    if k not in orig_sample: 
        print(k)

In [15]:
sd_sample.keys() == orig_sample.keys() 

True

In [16]:
count = 0 
total = 0 
tp_same_dialog_ids = [] 
for k, v in tp_test.items(): 
    sd_sample = v
    orig_sample = orig[k]
    len_sd = len(sd_sample['log'])
    len_orig = len(orig_sample['log'])
    assert len_sd== len_orig, (k, len_sd, len_orig)

    for idx, log in enumerate(sd_sample['log']): 
        assert log['metadata'] == orig_sample['log'][idx]['metadata']
        if idx % 2 == 0 and log['text'] != orig_sample['log'][idx]['text']: 
            count += 1 
        if idx % 2 == 0 and log['text'] == orig_sample['log'][idx]['text']: 
            tp_same_dialog_ids.append(f"{k}-{idx}")
        if idx % 2 == 0: 
            total += 1

        # assert log['span_info'] == orig_sample['log'][idx]['span_info']
        

count / total 


0.9584915897992403

In [17]:
len(tp_same_dialog_ids)

306

In [18]:
with open("data/laug_dst/TP/no_change_ids.txt", "w") as f: 
    for id in tp_same_dialog_ids: 
        f.write(id+"\n")

In [19]:
tp_same_dialog_ids

['PMUL3027.json-2',
 'MUL2305.json-2',
 'MUL0469.json-8',
 'MUL1024.json-6',
 'SNG0263.json-4',
 'MUL1350.json-2',
 'MUL1350.json-8',
 'MUL2405.json-2',
 'MUL2290.json-8',
 'MUL2162.json-12',
 'PMUL3596.json-4',
 'PMUL2755.json-4',
 'MUL1650.json-2',
 'PMUL3748.json-2',
 'PMUL3748.json-4',
 'MUL2177.json-16',
 'PMUL1276.json-12',
 'PMUL3890.json-10',
 'PMUL3890.json-12',
 'PMUL0286.json-4',
 'PMUL2124.json-14',
 'MUL1818.json-8',
 'PMUL2286.json-12',
 'SNG1048.json-2',
 'PMUL4388.json-0',
 'PMUL4388.json-4',
 'MUL1228.json-10',
 'MUL2637.json-12',
 'SNG01957.json-4',
 'PMUL3919.json-8',
 'PMUL1180.json-12',
 'PMUL3672.json-0',
 'PMUL3672.json-10',
 'SNG0345.json-6',
 'PMUL4255.json-14',
 'PMUL4131.json-12',
 'MUL0912.json-22',
 'MUL2012.json-14',
 'MUL2012.json-16',
 'PMUL2980.json-4',
 'MUL1546.json-10',
 'MUL2359.json-10',
 'PMUL0095.json-4',
 'PMUL3668.json-6',
 'PMUL2704.json-4',
 'PMUL3336.json-10',
 'MUL1883.json-18',
 'MUL0515.json-8',
 'MUL1800.json-6',
 'PMUL3558.json-0',
 'PM

In [20]:
sd_sample['log'][0].keys()


dict_keys(['text', 'originalText', 'metadata', 'span_info', 'dialog_act'])

In [21]:
orig_sample['log'][0].keys()

dict_keys(['dialog_act', 'metadata', 'span_info', 'text', 'turn_id'])

In [22]:
# for k in sd_sample: 
#     if sd_sample[k] != orig_sample[k]: 
#         print(k)
#         print(sd_sample[k])
#         print(orig_sample[k])

# create dataset with named entities replaced 

In [23]:
from parlai.tasks.multiwoz_dst_laug.mutator import EntityMutator

In [24]:
named_entity_slots_for_trippy ={
    "attraction-name",
    "restaurant-name",
    "hotel-name",
    "taxi-departure",
    "taxi-destination",
    "train-departure",
    "train-destination",
}

# for d in utt['metadata']:
# booked = utt['metadata'][d]['book']['booked']
# booked_slots = {}
# # Check the booked section
# if booked != []:
#     for s in booked[0]:
#         booked_slots[s] = normalize_label('%s-%s' % (d, s), booked[0][s]) # normalize labels
# # Check the semi and the inform slots
# for category in ['book', 'semi']:
#     for s in utt['metadata'][d][category]:
#         cs = '%s-book_%s' % (d, s) if category == 'book' else '%s-%s' % (d, s)
#         value_label = normalize_label(cs, utt['metadata'][d][category][s]) # normalize labels
#         # Prefer the slot value as stored in the booked section
#         if s in booked_slots:
#             value_label = booked_slots[s]
#         # Remember modified slots and entire dialog state
#         if cs in slot_list and cumulative_labels[cs] != value_label:
#             modified_slots[cs] = value_label
#             cumulative_labels[cs] = value_label

In [25]:
import numpy as np  
from argparse import ArgumentParser

In [26]:
SEED = 42
rng = np.random.RandomState(SEED)
mutator = EntityMutator(opt=None, rng=rng)

In [27]:
from pprint import pprint

In [28]:
a = {"b": {"c": "d"}}

In [29]:
x = a['b']
x['c'] = "x"

a

{'b': {'c': 'x'}}

In [30]:
sample =    {'dialog_act': {'Booking-Request': [['Stay', '?'], ['Day', '?']]},
    'metadata': {'attraction': {'book': {'booked': []},
      'semi': {'area': '', 'name': '', 'type': ''}},
     'hospital': {'book': {'booked': []}, 'semi': {'department': ''}},
     'hotel': {'book': {'booked': [],
       'day': 'monday',
       'people': '',
       'stay': ''},
      'semi': {'area': 'centre',
       'internet': 'dontcare',
       'name': 'alexander bed and breakfast',
       'parking': 'not mentioned',
       'pricerange': 'not mentioned',
       'stars': 'not mentioned',
       'type': 'guesthouse'}},
     'police': {'book': {'booked': []}, 'semi': {}},
     'restaurant': {'book': {'booked': [],
       'day': '',
       'people': '',
       'time': ''},
      'semi': {'area': '', 'food': '', 'name': '', 'pricerange': ''}},
     'taxi': {'book': {'booked': []},
      'semi': {'arriveBy': '',
       'departure': '',
       'destination': '',
       'leaveAt': ''}},
     'train': {'book': {'booked': [], 'people': ''},
      'semi': {'arriveBy': '',
       'day': '',
       'departure': '',
       'destination': '',
       'leaveAt': ''}}},
    'span_info': [],
    'text': 'what day would you like to book on and for how long ?',
    'turn_id': 5}

In [31]:
def replace_dict_values_str(obj, orig_slot, new_slot): 
    # obj['attraction']['semi']['area'] = "hi"
    # return 
    # print(obj)
    if isinstance(obj, dict): 
        for k, v in obj.items(): 
            obj[k] = replace_dict_values_str(v, orig_slot, new_slot)
    elif isinstance(obj, str): 
        obj = obj.replace(orig_slot, new_slot)
    elif isinstance(obj, list): 
        for idx in range(len(obj)): 
            obj[idx] = replace_dict_values_str(obj[idx], orig_slot, new_slot)    
    return obj 

In [32]:
# replace_dict_values_str(sample, "alexander bed and breakfast", "some sample word")
replace_dict_values_str(sample, "alexander bed and breakfast", "whatever")
sample

{'dialog_act': {'Booking-Request': [['Stay', '?'], ['Day', '?']]},
 'metadata': {'attraction': {'book': {'booked': []},
   'semi': {'area': '', 'name': '', 'type': ''}},
  'hospital': {'book': {'booked': []}, 'semi': {'department': ''}},
  'hotel': {'book': {'booked': [], 'day': 'monday', 'people': '', 'stay': ''},
   'semi': {'area': 'centre',
    'internet': 'dontcare',
    'name': 'whatever',
    'parking': 'not mentioned',
    'pricerange': 'not mentioned',
    'stars': 'not mentioned',
    'type': 'guesthouse'}},
  'police': {'book': {'booked': []}, 'semi': {}},
  'restaurant': {'book': {'booked': [], 'day': '', 'people': '', 'time': ''},
   'semi': {'area': '', 'food': '', 'name': '', 'pricerange': ''}},
  'taxi': {'book': {'booked': []},
   'semi': {'arriveBy': '',
    'departure': '',
    'destination': '',
    'leaveAt': ''}},
  'train': {'book': {'booked': [], 'people': ''},
   'semi': {'arriveBy': '',
    'day': '',
    'departure': '',
    'destination': '',
    'leaveAt': 

In [33]:
import json

In [34]:
# orig_fn = "/data/home/justincho/trippy-public-master/data/MULTIWOZ2.3/test_dials.json"
orig_fn = "/data/home/justincho/trippy-public-master/data/MULTIWOZ2.3/data.json"
with open(orig_fn, "r") as f: 
    orig = json.load(f)

In [79]:
from tqdm import tqdm 

In [133]:
with open("/data/home/justincho/trippy-public-master/data/MULTIWOZ2.3/testListFile.json", "r") as f: 
    test_convs = f.readlines()
    test_convs = [t.replace("\n", "") for t in test_convs]

In [134]:
test_convs[0]

'MUL0484.json'

In [183]:
swapped_ct = 0 
change = False 
nei_conversion_map = {} 
nei_orig = orig.copy()  
slot_vals = set()
change_list = [] 
n_text_examples = 0 
print_ = True 
for dial_id, dial in tqdm(nei_orig.items()): 
    # print(k)
    # break

    def replace_with_scrambled_entity(domain, slot_type, slot_value, print_=False): 
        slot_value = slot_value.lower() 
        change = False 
        if f"{domain}-{slot_type}" in named_entity_slots_for_trippy: 
            change = True

            if "".join(sorted(slot_value)) not in nei_conversion_map: 
                new_v  = mutator.scramble(slot_value)
                nei_conversion_map["".join(sorted(slot_value))] = new_v
            else: 
                new_v = nei_conversion_map["".join(sorted(slot_value))]

            # go through all turns and replace if found 
            for idx2 in range(len(dial['log'])): 
                # update only if the golden slot value is actually found in the text
                if slot_value.lower() in dial['log'][idx2]['text'].lower(): 
                    # update text 
                    dial['log'][idx2]['text']= dial['log'][idx2]['text'].lower().replace(slot_value, new_v)
                    # update metadata 
                    replace_dict_values_str(dial['log'][idx2]['metadata'], slot_value, new_v)
                    change = True 

                    if print_: 
                        print(dial['log'][idx2]['text'])
                        print(f"{domain} {slot_type} {slot_value}")
                        print(f"{domain} {slot_type} {new_v}")
                
        return change 
    
    for idx in reversed(range(0, len(dial['log']), 2)): 
        user_utt = dial['log'][idx]['text']
        sys_utt = dial['log'][idx+1]['text']
        metadata = dial['log'][idx+1]['metadata']
        # print(idx)
        # print(user_utt)
        # print(sys_utt)
        # pprint(metadata)
        change = False 
        for domain, slots in metadata.items(): 
            for k, v in slots['book'].items(): 
                if k == "booked": 
                    if v == []: 
                        continue 
                    else: 
                        for booked_places in slots['book']['booked']: 
                            for slot_type, slot_value in booked_places.items():
                                slot_vals.add(slot_value)
                                change = change or replace_with_scrambled_entity(domain, slot_type, slot_value, print_=print_)
                                
                        
                elif v and v!= "not mentioned":  
                    # print(f"{domain} {k} {v}")
                    change = change or replace_with_scrambled_entity(domain, k, v, print_=print_)

            for k,v in slots['semi'].items(): 
                if v and v!="not mentioned": 
                    change = change or replace_with_scrambled_entity(domain, k, v, print_=print_)
        n_text_examples += dial_id in test_convs 
        if change and dial_id in test_convs:
            dial_full_id = f"{dial_id.lower().replace('.json', '')}-{idx//2}"
            change_list.append(dial_full_id) 

    # if True:
    # # if change: 
    #     pprint(nei_conversion_map)
    #     for idx2 in range(len(dial['log'])):
    #         print(f"{idx2}: {dial['log'][idx2]['text']}")
    #         if idx2 % 2: 
    #             pprint(dial['log'][idx2]['metadata'])
    #     break 
    

100%|██████████| 10438/10438 [00:09<00:00, 1091.46it/s]


In [184]:
assert n_text_examples == 7372 

In [185]:
sorted(change_list)[0]

'mul0003-3'

In [186]:
with open("/data/home/justincho/trippy-public-master/data/laug_dst/NEI/changed_dialids_test_False.txt", "r") as f: 
    changed_parlai_ids = f.readlines()
    changed_parlai_ids = [c.replace("\n", "") for c in changed_parlai_ids]

In [187]:
sorted(change_list)[0], sorted(changed_parlai_ids)[0]

('mul0003-3', 'mul0003-5')

In [188]:
len(changed_parlai_ids), len(change_list), len(set(change_list).intersection(changed_parlai_ids))

(5827, 803, 767)

In [81]:
nei_conversion_map

{'  abbcdeeefghilmrrty': 'ahmed ylr ergitbbfec',
 'abcdegimr': 'reamcgbdi',
 '  abeeeghiimmnnrrsttw': ' rtnewmemshgn eatibri',
 'abeelllln': 'llelleanb',
 'ely': 'eyl',
 '    aabbcddeeeghiiilmnnoprrssxyy': 'medesb lbon eryxiyncairahdspg  i',
 '  aceeghnoorsstuu': 'stoc norsegeuau h',
 'chinorw': 'rcinwoh',
 '  aaabeeemnrrrsttuz': 'urata zeeantmrs ber',
 '  aacchhillnrsstu': 'hlalccs htunairs ',
 '  abeegghhllnooorrstuuy': 'bsnhooyerg ehrlo gluuat',
 'beeghooprrtu': 'grrtopbheuoe',
 ' aadijnoortt': 'nodaarti jot',
 '  effhknnosuuuy': 'onyeffuhku un s',
 'enno': 'nnoe',
 '     abbccdeehinnnorrttty': 'ioetbrnd nc a c nrtytebh ',
 ' bdfhioooprrssstt': 'srbsoitdfrspot oh',
 ' aacehhorsuwy': 'w sayaehruhco',
 ' ehhoorstuw': 'ureotsohh w',
 ' acdegnrrruy': 'ranyrcrgu de',
 '  aceeeggghhhikllllnossu|': 'lgloa hgnh|cselk hiesguel',
 '     aabbcddeeeeghhiiilmnnoprrsstxyy': ' eipbnontyagymrlx ca isibs hhre deed',
 ' cceeghillorst': ' lhgotcresclei',
 '   acceehiinprtttuyzz': 'ipizce cntt uhatz yr

In [90]:
ne_samples = [
    "birmingham new street", 
    "golden house", 
    "express by holiday inn cambridge",
    "the cambridge belfry"
]

for ne_sample in ne_samples: 
    print("".join(sorted(ne_sample.lower())) in nei_conversion_map)

True
True
True
True


In [38]:
#need to update the dialogue_acts.json file as well 

with open("data/laug_dst/SD/dialogue_acts.json", "r") as f: 
    dacts = json.load(f)

In [87]:
load_acts("data/laug_dst/SD/dialogue_acts.json")

{('SNG01856.json', '2', 'hotel-pricerange'): ['cheap'],
 ('SNG01856.json', '2', 'hotel-choice'): ['1'],
 ('SNG01856.json', '2', 'hotel-type'): ['hotel'],
 ('SNG01856.json', '4', 'booking-ref'): ['7gawk763'],
 ('SNG0129.json', '1', 'police-addr'): ['parkside , cambridge'],
 ('SNG0129.json', '1', 'police-phone'): ['01223358966'],
 ('SNG0129.json', '1', 'police-name'): ['parkside police station'],
 ('SNG0129.json', '2', 'police-post'): ['cb11jg'],
 ('SNG0129.json', '2', 'police-name'): ['parkside police station'],
 ('MUL2168.json', '1', 'train-choice'): ['over 1,000'],
 ('MUL2168.json', '3', 'train-arriveBy'): ['20:23'],
 ('MUL2168.json', '3', 'train-leaveAt'): ['17:40'],
 ('MUL2168.json', '3', 'train-day'): ['wednesday'],
 ('MUL2168.json', '3', 'train-departure'): ['birmingham new street'],
 ('MUL2168.json', '5', 'hotel-name'): ['the cambridge belfry'],
 ('MUL2168.json', '7', 'booking-ref'): ['5nawgjdc'],
 ('SNG01445.json', '1', 'police-addr'): ['parkside , cambridge'],
 ('SNG01445.json'

In [85]:
ACTS_DICT = {'taxi-depart': 'taxi-departure',
             'taxi-dest': 'taxi-destination',
             'taxi-leave': 'taxi-leaveAt',
             'taxi-arrive': 'taxi-arriveBy',
             'train-depart': 'train-departure',
             'train-dest': 'train-destination',
             'train-leave': 'train-leaveAt',
             'train-arrive': 'train-arriveBy',
             'train-people': 'train-book_people',
             'restaurant-price': 'restaurant-pricerange',
             'restaurant-people': 'restaurant-book_people',
             'restaurant-day': 'restaurant-book_day',
             'restaurant-time': 'restaurant-book_time',
             'hotel-price': 'hotel-pricerange',
             'hotel-people': 'hotel-book_people',
             'hotel-day': 'hotel-book_day',
             'hotel-stay': 'hotel-book_stay',
             'booking-people': 'booking-book_people',
             'booking-day': 'booking-book_day',
             'booking-stay': 'booking-book_stay',
             'booking-time': 'booking-book_time',
}


In [86]:
def load_acts(input_file):
    with open(input_file) as f:
        acts = json.load(f)
    s_dict = {}
    for d in acts:
        for t in acts[d]:
            # Only process, if turn has annotation
            if isinstance(acts[d][t], dict):
                for a in acts[d][t]:
                    aa = a.lower().split('-')
                    if aa[1] == 'inform' or aa[1] == 'recommend' or aa[1] == 'select' or aa[1] == 'book':
                        for i in acts[d][t][a]:
                            s = i[0].lower()
                            v = i[1].lower().strip()
                            if s == 'none' or v == '?' or v == 'none':
                                continue
                            slot = aa[0] + '-' + s
                            if slot in ACTS_DICT:
                                slot = ACTS_DICT[slot]
                            key = d + '.json', t, slot
                            # In case of multiple mentioned values...
                            # ... Option 1: Keep first informed value
                            if key not in s_dict:
                                s_dict[key] = list([v])
                            # ... Option 2: Keep last informed value
                            #s_dict[key] = list([v])
    return s_dict

In [69]:
from collections import Counter 
slot_counter = Counter()
slotname_counter = Counter()
for dial_id, turn_acts in dacts.items(): 

    for turn, acts in turn_acts.items(): 

        if isinstance(acts, dict): 
            for slot, slot_vals in acts.items(): 
                for sv in slot_vals: 
                    # print(slot, sv)
                    slot_counter[f"{slot}-{sv[0]}-{sv[1]}"] += 1 
                    slotname_counter[f"{slot}-{sv[0]}"] += 1 

                    if f"{slot}-{sv[0]}" == "Restaurant-Inform-Name" and sv[1] != "None": 
                        assert "".join(sorted(sv[1].lower())) in nei_conversion_map, sv[1]
        else: 
            if acts == "No Annotation": 
                continue 
            print(f"Not a dict: {acts}")

    # print(acts)
    # break

AssertionError: pizza express Fed Ditton

In [43]:
orig_fn

'/data/home/justincho/trippy-public-master/data/MULTIWOZ2.3/data.json'

In [44]:
nei_orig_fn = "/data/home/justincho/trippy-public-master/data/laug_dst/NEI/data.json"
with open(nei_orig_fn, "w") as f: 
    json.dump(nei_orig, f, indent=4)

In [45]:
orig_fn_test = "/data/home/justincho/trippy-public-master/data/MULTIWOZ2.3/test_dials.json"


In [46]:
with open(orig_fn_test, "r") as f: 
    orig_test_data = json.load(f)

In [47]:
list(nei_orig.keys())[0], list(orig_test_data.keys())[0]

('SNG01856.json', 'MUL0003.json')

In [48]:
nei_test_data = {} 
for k in orig_test_data.keys(): 
    nei_test_data[k] = nei_orig[k]

In [49]:
len(nei_test_data)

1000

In [50]:
nei_test_fn = "data/laug_dst/NEI/test_dials.json"
with open(nei_test_fn, "w") as f: 
    json.dump(nei_test_data, f, indent=4)

In [51]:
nei_test_data['MUL0003.json']['log']

[{'text': "i 'm looking for a place to stay . it needs to be a guesthouse and include free wifi .",
  'metadata': {},
  'dialog_act': {'Hotel-Inform': [['Type', 'guesthouse'],
    ['Internet', 'yes']]},
  'span_info': [['Hotel-Inform', 'Type', 'guesthouse', 14, 14]],
  'turn_id': 0},
 {'text': 'there are 23 hotels that meet your needs . would you like to narrow your search by area and/or price range ?',
  'metadata': {'taxi': {'book': {'booked': []},
    'semi': {'leaveAt': '',
     'destination': '',
     'departure': '',
     'arriveBy': ''}},
   'police': {'book': {'booked': []}, 'semi': {}},
   'restaurant': {'book': {'booked': [], 'people': '', 'day': '', 'time': ''},
    'semi': {'food': '', 'pricerange': '', 'name': '', 'area': ''}},
   'hospital': {'book': {'booked': []}, 'semi': {'department': ''}},
   'hotel': {'book': {'booked': [], 'people': '', 'day': '', 'stay': ''},
    'semi': {'name': 'not mentioned',
     'area': 'not mentioned',
     'parking': 'not mentioned',
     

In [52]:
import pandas as pd 

In [53]:
df = pd.read_csv("/data/home/justincho/trippy-public-master/results/of_interest/multiwoz23_lr1e-4_2022-01-05_09:41:24_fewshot_False_46/pred_res.test.finalNEI.csv")

In [54]:
NAMED_ENTITY_SLOTS_FOR_TRIPPY ={
    "attraction-name",
    "restaurant-name",
    "hotel-name",
    "taxi-departure",
    "taxi-destination",
    "train-departure",
    "train-destination",
}


In [55]:
from metric_bert_dst import load_dataset_config, tokenize, check_slot_inform

In [93]:
dataset_config = "dataset_config/multiwoz21.json"
class_types, slots, label_maps = load_dataset_config(dataset_config)

In [96]:
from collections import Counter

In [130]:
jgas = [] 
ne_jga =[]
ct =0 
false_pred_ct = 0 
false_pred_dict= {} 
counter = Counter() 
print_ct =  0 
for idx, row in df.iterrows(): 
    found_ne_slot = False
    for ne_slot in NAMED_ENTITY_SLOTS_FOR_TRIPPY: 
        if row[f"{ne_slot}_gold"] != "none": 
            found_ne_slot = True 
            break

    if found_ne_slot: 
        jga = int(all([row[f"{slot}_gold"] == row[f"{slot}_pred"] for slot in slots]))
        jgas.append(jga)

    for ne_slot in NAMED_ENTITY_SLOTS_FOR_TRIPPY: 
        # if row[f"{ne_slot}_gold"] != "none":
        #     continue
        # if row[f"{ne_slot}_gold"] == "none" and row[f"{ne_slot}_pred"] != "none": 
        #     # print(f"{}")
        #     continue
        if row[f"{ne_slot}_gold"] != "none" and row[f"{ne_slot}_pred"] == row[f"{ne_slot}_gold"] : 
            # print(row[f"{ne_slot}_gold"])
            # print(row[f"{ne_slot}_pred"])
            counter[row[f"{ne_slot}_gold"]] +=1 
            ct += 1 
        if row[f"{ne_slot}_gold"] != "none" and row[f"{ne_slot}_pred"] != "none" and row[f"{ne_slot}_pred"] != row[f"{ne_slot}_gold"] : 
            if print_ct < 100: 
                print(f"{row['id']} \n\tpred: " + row[f"{ne_slot}_pred"], "\n\tgold: " + row[f"{ne_slot}_gold"])
                print_ct += 1 
            
        if row[f"{ne_slot}_gold"] == "none" and row[f"{ne_slot}_pred"] != "none": 
            false_pred_ct += 1 
            # print()


np.mean(jgas), len(jgas), ct, false_pred_ct  
    # are there many casese where there were no named entities yet one was predicted for in the augmented set? 

mul0011.json-2 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-3 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-4 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-5 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-6 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-7 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-8 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-9 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-10 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-11 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-11 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-12 
	pred: cafe 
	gold: lcoafns wec
mul0011.json-12 
	pred: cafe 
	gold: lcoafns wec
mul0014.json-4 
	pred: lrhtaa ein 
	gold: arianl
mul0014.json-5 
	pred: lrhtaa ein 
	gold: arianl
mul0021.json-3 
	pred: the oehkc niaa t nzpicdrwza b [ SEP ] absolutely ! enlf etoilhhestd le is a [ UNK ] star hotel and their phone number is 01 223 355017 . is there anything else i can help you with ? [ SEP ] i ' d just need their star rating and phone number , thank y

(0.04690338563170933, 6055, 2275, 71)

In [114]:
from pprint import pprint

In [127]:
def show_concise_metadata(metadata_dict): 
    concise = {} 
    for domain, slots in metadata_dict.items(): 
        concise_domain = {} 
        for slot_type, slotkv in slots.items(): 
            concise_slotkv = {} 
            for k,v in slotkv.items(): 
                if v: 
                    concise_slotkv[k] = v 
            if concise_slotkv: 
                concise_domain[slot_type] = concise_slotkv 
        if concise_domain: 
            concise[domain] = concise_domain 
    if concise: 
        return concise 

In [129]:
for t in nei_test_data["MUL0594.json"]["log"]: 
    print(t["text"])
    # pprint(t["metadata"])
    if t["metadata"]: 
        pprint(show_concise_metadata(t["metadata"]))

hello , i need to book a ticket going to reamcgbdi arriving by 16:30 , can you help me with that ?
there are a lot of trains available . where are you departing from ?
{'train': {'semi': {'arriveBy': '16:30',
                    'day': 'not mentioned',
                    'departure': 'not mentioned',
                    'destination': 'reamcgbdi',
                    'leaveAt': 'not mentioned'}}}
i 'm departing from norurobbex on friday .
tr2145 looks like it will get you there with plenty of time to spare .
{'train': {'semi': {'arriveBy': '16:30',
                    'day': 'friday',
                    'departure': 'norurobbex',
                    'destination': 'reamcgbdi',
                    'leaveAt': 'not mentioned'}}}
thanks , could you let me know the departure and arrival time ?
of course . it was actually the tr2144 , my apologies about that .. it leaves at 14:32 and gets you there at about 15:32 . it costs 17.90 pounds .
{'train': {'semi': {'arriveBy': '16:30',
          

In [58]:
counter.most_common(10)

[('deirmgbca', 1317),
 ('ely', 241),
 ('prbeturooegh', 61),
 ('ogl ni ndronkcsoss', 55),
 ('gvtaenees', 35),
 ('ceetelris', 25),
 ('nodsna', 25),
 ('ahjalamr', 21),
 ('zsahir', 20),
 ('avnoal', 18)]